In [1]:
import sys
sys.path.append('../')

from warnings import filterwarnings
filterwarnings('ignore')

from src.utils.video import VideoIterator

import numpy as np
import cv2
from yolo5face.get_model import get_model
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
model = get_model("yolov5n", gpu=-1, target_size=512, min_face=24)

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [3]:
input_file_path = '../data/Learn English With Street Interviews! #40 What Makes you Happy_#britishenglish #howtospeakenglish.mp4'
output_file_path = '../data/output2.mp4'

video_iterator = VideoIterator(input_file_path)
video_writer = cv2.VideoWriter(
    output_file_path,
    cv2.VideoWriter_fourcc(*'MP4V'),
    video_iterator.fps,
    (video_iterator.width, video_iterator.height),
)

for frame in tqdm(video_iterator):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes = model(frame)[0][0]

    if len(boxes) > 0:
        mask = np.zeros_like(frame) - 1
        for box in boxes:
            mask[box[1]:box[3], box[0]:box[2]] = 0

        blurred_frame = cv2.GaussianBlur(frame, (45, 45), 0)
        frame = np.where(
            mask==np.array([255, 255, 255]),
            frame,
            blurred_frame
        )
    
    video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
video_writer.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


  0%|          | 0/6860 [00:00<?, ?it/s]

In [7]:
from moviepy.editor import VideoFileClip

In [8]:
input_file_path = '../data/Learn English With Street Interviews! #40 What Makes you Happy_#britishenglish #howtospeakenglish.mp4'
output_file_path = '../data/output2.mp4'

In [9]:
input_clip = VideoFileClip(input_file_path)
output_clip = VideoFileClip(output_file_path)

In [10]:
output_clip = output_clip.set_audio(input_clip.audio)

In [11]:
output_clip.write_videofile(
    '../data/final2.mp4',
    codec='libx264', 
    audio_codec='aac', 
    temp_audiofile='temp-audio.m4a', 
    remove_temp=True,
)

Moviepy - Building video ../data/final2.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video ../data/final2.mp4



Moviepy - Done !
Moviepy - video ready ../data/final2.mp4
